<a href="https://colab.research.google.com/github/MaysoonHelmy/Guest-Satisfaction-Prediction/blob/main/Guest_Satisfaction_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_csv("/content/GuestSatisfactionPrediction.csv")

<ipython-input-2-0c0a2d05ca28>:1: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/GuestSatisfactionPrediction.csv")


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8724 entries, 0 to 8723
Data columns (total 69 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                8724 non-null   int64  
 1   listing_url                       8724 non-null   object 
 2   name                              8724 non-null   object 
 3   summary                           8497 non-null   object 
 4   space                             7105 non-null   object 
 5   description                       8630 non-null   object 
 6   neighborhood_overview             6490 non-null   object 
 7   notes                             5215 non-null   object 
 8   transit                           5975 non-null   object 
 9   access                            6011 non-null   object 
 10  interaction                       6164 non-null   object 
 11  house_rules                       6754 non-null   object 
 12  thumbn

In [ ]:
df.columns

Index(['id', 'listing_url', 'name', 'summary', 'space', 'description',
       'neighborhood_overview', 'notes', 'transit', 'access', 'interaction',
       'house_rules', 'thumbnail_url', 'host_id', 'host_url', 'host_name',
       'host_since', 'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_has_profile_pic',
       'host_identity_verified', 'street', 'neighbourhood',
       'neighbourhood_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'square_feet',
       'nightly_price', 'price_per_stay', 'security_deposit', 'cleaning_fee',
       'guests_included', 'extra_people', 'minimum_nights', 'maximum_nights',

In [ ]:
df.head()

,id,listing_url,name,summary,space,description,neighborhood_overview,notes,transit,access,...,number_of_stays,first_review,last_review,review_scores_rating,requires_license,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification
0,21514496,https://www.airbnb.com/rooms/21514496,PRIVATE BEDROOM DOWNTOWN 2 BED/ 2 BATH #5,Beautiful 2 bedroom 2 bathroom furnished apart...,Each apartment is fully furnished and consists...,Beautiful 2 bedroom 2 bathroom furnished apart...,NaN,***Note that the pictures on the website may o...,NaN,NaN,...,2,11/17/2017,11/17/2017,80.0,f,f,f,strict_14_with_grace_period,f,f
1,990185,https://www.airbnb.com/rooms/990185,4 bed/2 ba Family Retreat in SD,We feel our house is a great place for a famil...,Our house is ideal for large families or group...,We feel our house is a great place for a famil...,Neighborhood Mira Mesa is a culturally diverse...,NaN,NaN,Our guests may use the BBQ grill.,...,26,8/2/2013,7/31/2019,97.0,f,f,f,moderate,f,f
2,19878244,https://www.airbnb.com/rooms/19878244,San Diego Dream Villa,"Zen, Luxury and the Best Location in Americas ...","San Diego Dream Villa, this unique and luxurio...","Zen, Luxury and the Best Location in Americas ...",This amazing house is located within a few min...,Im always available to make your trip experien...,"If you are bringing your car, remember that SD...",A couple of days before your arrival you will ...,...,170,7/23/2017,7/16/2019,98.0,f,f,f,strict_14_with_grace_period,f,f
3,24561458,https://www.airbnb.com/rooms/24561458,Studio with Piazza View in Downtown Little Italy,Start the day with breakfast on the sunny pati...,"Newly built, this beautiful studio is located ...",Start the day with breakfast on the sunny pati...,"Spend the day at the zoo in Balboa Park, or ex...",The apartment is located on the Piazza Della F...,"MTS, Train, and Trolly are close by. The airpo...",You will not be sharing the apartment with any...,...,212,5/1/2018,8/3/2019,98.0,f,t,f,moderate,f,f
4,32269829,https://www.airbnb.com/rooms/32269829,Beachside Retreat w/ Private Rooftop Deck w/ V...,This is a 2 bedroom/1 bathroom beachside unit...,Beach living at its finest! The highlight of ...,This is a 2 bedroom/1 bathroom beachside unit...,Mission Beach is an amazing coastal community ...,NaN,The Boardwalk and Bayside Walk are wonderful w...,This is a private home that is part of a typic...,...,2,6/30/2019,6/30/2019,100.0,f,t,f,strict_14_with_grace_period,f,f


In [ ]:
df.neighborhood_overview.value_counts()

,count
neighborhood_overview,
South Mission,45
North Mission,43
Activities Amusement parks Antiquing Basketball Beachcombing Biking Bird watching Boating Botanical gardens Casino gambling Fishing Golf Hiking Horseback riding Horse racing Hot air ballooning Ice skating Jet skiing Kayaking Kite surfing Live theatre Movies Mountain biking Outlet shopping Paddle boating Paragliding Parasailing Rock climbing Sailing Scuba diving Skating Shopping Sightseeing Snorkeling Surfing Swimming Tennis Walking Water skiing Water tubing Whale watching Windsurfing Local Businesses and Services ATM / banks Beauty spas Churches Duty free shopping Ecotourism Fitness gyms Groceries Health spas Hospitals Laundromats LegoLand Libraries Malls Marinas Medical services Massage services Museums Parks Playgrounds Restaurants Scenic drives SeaWorld Synagogues Water parks Winery tours Wildlife park Zoo,34
Mission Valley highlights - 6 minute walk to Lazy Dog Restaurant & Bar (global comfort cuisines) - 1 mile to The Broken Yolk Cafe (cheery digs offering all-day breakfast) - 5 minute walk to Trader Joe's,24
"Petco Park, Gaslamp Quarter, Balboa Park, San Diego Zoo, Sea World San Diego, Convention Center, Little Italy, La Jolla Cove, Mission Beach, Oceanside, and MUCH MUCH MORE!!",20
...,...
"One of the best neighborhoods within San Diego. Within blocks of one of the best beaches for beach goers, especially families with kids. Also within 3-4 blocks of great restaurants and shops.",1
We live in a residential beach neighbor lined with palm trees with views of the beach streaming from Ocean beach all the way to La Jolla. You are only 2 minutes away from an adorable beach town called Birdrock and Soledad Mountain lays right behind you.,1
"The location of the suite is one of its best attributes. It's just blocks away from Downtown Gaslamp and a short ride to Hillcrest, North Park, South Park, and Coronado. It's a family neighborhood and safe to live.",1


In [ ]:
df.description.value_counts()

,count
description,
"Featured in The New York Times, The Wall Street Journal, and Fast Company, we’ve hosted over 350k guests in 2,500+ spaces with an average rating of 4.6 stars. Each space is designed and run by us, so you’ll always know what to expect from your stay. At Sonder, we’ve combined the best parts of home and hotel. - 24/7 support via text, email, or phone - Coffee, fresh towels, and bathroom essentials provided - Hotel-standard cleaning before you arrive - Self check-in - Private balcony - In-suite laundry - Fully-equipped kitchen - One garage parking space Building amenities - Hot tub and pool - Fitness center - BBQ facilities - Pool table Mission Valley highlights - 6 minute walk to Lazy Dog Restaurant & Bar (global comfort cuisines) - 1 mile to The Broken Yolk Cafe (cheery digs offering all-day breakfast) - 5 minute walk to Trader Joe's Getting around - 4 minute drive to Mission Valley Center lightrail station - 11 minute drive to Downtown - 16 minute drive to SAN Airport Mission Valley hi",24
"Featured in The New York Times, The Wall Street Journal, and Fast Company, we’ve hosted over 350k guests in 2,500+ spaces with an average rating of 4.6 stars. Each space is designed and run by us, so you’ll always know what to expect from your stay. At Sonder, we’ve combined the best parts of home and hotel. - 24/7 support via text, email, or phone - Coffee, fresh towels, and bathroom essentials provided - Hotel-standard cleaning before you arrive - Self check-in - Private balcony - In-suite laundry - Fully-equipped kitchen - Free garage parking Building amenities - Outdoor pool - Hot tub - Fitness center - BBQ facilities Downtown highlights - Walkability score - 95 - 4 minute walk to The Mission (Asian & Latin American flavored cuisines) - 8 minute walk to Lolita's Mexican Food (best Mexican classics) Getting around - 3 blocks to Market St & 14th St bus stop - 10 minute drive to Coronado Beach - 15 minute drive to SAN Airport Downtown highlights - Walkability score - 95 - 4 minute w",16
"California Dreams Hostel is a hostel on the shores of the Pacific Ocean, in Southern California, a hostel ready to become your home for the time exploring California. Amenities: Reception, Communal Lounging Areas, Efficiently Designed Kitchen Facilities, Showers and Bathrooms, Free WiFi and Luggage Storage. Located in the best San Diego neighborhood for exploring the local attractions, it offers safety, cleanness, comfort and most of all a memorable friendly experience.",16
"Gorgeous 1BD/1BA Unit in the heart of San Diego - Downtown. Professionally managed and fully stocked w/ everything from salt, pepper, shampoo and body-wash to a sparkling water welcome gift! We are available to assist you with absolutely anything you need through midnight, 7 days a week We offer rewards and complimentary upgrades to our regular repeat guests. We individually design each room that we have in the building so, while your room will have similar furnishings and features as the model pictured, there may be slight variations in your unit from the photos.",14
"Gorgeous Studio Unit in the heart of San Diego - Downtown. Professionally managed and fully stocked w/ everything from salt, pepper, shampoo and body-wash to a sparkling water welcome gift! We are available to assist you with absolutely anything you need through midnight, 7 days a week We offer rewards and complimentary upgrades to our regular repeat guests. We individually design each room that we have in the building so, while your room will have similar furnishings and features as the model pictured, there may be slight variations in your unit from the photos.",11
...,...
"Presenting our beautiful new offering available just in time for Summer 2019! Live the La Jolla lifestyle in this stunning new construction single family house located blocks to LJ Shores beach. This premier end unit is located in an enclave of 6 new construction homes & features 2bd+2.5ba, 3 outdoor areas, 3 levels, & sleeps 6.

In [ ]:
df.space.value_counts()

,count
space,
- Private balcony - In-suite laundry - Fully-equipped kitchen - One garage parking space Building amenities - Hot tub and pool - Fitness center - BBQ facilities - Pool table Mission Valley highlights - 6 minute walk to Lazy Dog Restaurant & Bar (global comfort cuisines) - 1 mile to The Broken Yolk Cafe (cheery digs offering all-day breakfast) - 5 minute walk to Trader Joe's Getting around - 4 minute drive to Mission Valley Center lightrail station - 11 minute drive to Downtown - 16 minute drive to SAN Airport,24
#NAME?,19
- Private balcony - In-suite laundry - Fully-equipped kitchen - Free garage parking Building amenities - Outdoor pool - Hot tub - Fitness center - BBQ facilities Downtown highlights - Walkability score - 95 - 4 minute walk to The Mission (Asian & Latin American flavored cuisines) - 8 minute walk to Lolita's Mexican Food (best Mexican classics) Getting around - 3 blocks to Market St & 14th St bus stop - 10 minute drive to Coronado Beach - 15 minute drive to SAN Airport,16
"JUST OPENED 3.27.19 AFTER EXTENSIVE 14 MONTH REMODEL! - It pleases us to share with you what our guests are saying… Property has several size units, so please check the reviews in Context and check that the unit that you are interested in has capacity for your number of guests…. “Very nice 3 bedroom place in a nice area. Very comfortable for a family of 6. Very close to Sea World. Check in and out was very easy.” - Al (April 2019) “Great and stylish place. Wish we could’ve stayed longer but the room was very clean and new. Close to everything we wanted to enjoy in San Diego” - Sylvester (April 2019) “Excellent location if you're planning to travel to Sea World.” - Enrique (April 2019) “Amazing location and price. You can’t beat it. The unit is completely remodeled with all upscale features including cabinets, hardware, flooring, backsplash, front loading washer/dryer, and mounted televisions. The main bed was extremely comfortable and the unit was spotless. I will most definitely stay",10
"You'll be in an urban up and coming neighborhood with working families and a lively main avenue one block over so there is a possibility you will hear various street noises during the day and at night. We offer amenities such as: Keurig Coffee maker with some starter coffee pods, High speed WiFi, Samsung Smart TVs with available access to Amazon and Netflix streaming (*with a valid log in). There is a fully functioning kitchen with full sized stainless steel appliances, fridge, microwave, dishwasher and toaster-oven. Dishes, glassware, silverware, pots and pans are also provided. Beds have fresh linen, towels are provided, and we offer body soap and shampoo and conditioner. There is an iron and ironing board, and although San Diego has mild weather year round, the home offers both heat/AC in the living room, personal heaters in each room for the cooler nights, and ceiling fans in every room for the warmer ones. CHECK IN: There is keyless entry with a personal guest code, which allow",8
...,...
"If you’re traveling to San Diego, why not experience California living at its finest? This beautiful, three bedroom (with bonus studio), two bathroom open floor plan home sits on a large, peaceful canyon. And while you’ll feel like you’ve gotten away from it all, you’re just a short drive to all of San Diego’s famous attractions. 5 miles to San Diego Zoo; 10 miles to SeaWorld and Airport. This unique home is a sanctuary for those seeking a peaceful escape from their hectic lives. The large patio has a calming water feature that beckons you to relax and enjoy the view. Another patio is perfect for a quiet dinner, complete with BBQ grill and seating for four. As you venture into the hillside yard, you can unwind in the private hot tub while enjoying the lush landscaping that surrounds you. There are large windows throughout, including five sets of french doors, that flood the house with natural light. The home remains private with gorgeous views to the c